# Practical 2 - Survival analysis
****
In this practical session, using the same data as practical 1, we will explore some of the concepts covered in the lectures and give you the opportunity to run some code.  


In this practical session you will explore how to fit a prediction model for time to event data.

We will use the same data set used for practical 1 but rather than modelling if the event happened in 2 years, we will consider any event during the whole follow up in the data (9.5 years) and the time at which it occured.  

Due to limited time, we will only consider cox proportional hazard models as they are one of the most common types of survival models used and we will not cover checking the models assumptions, such as proportional hazards, and what to do if the assumptions do not hold.

Note: Other models, including non-parametric or weibull survival models could be more appropriate to use depending on your data and this is something that should also be considered and tested when developing a prediction model.



### Lets begin

First lets load the packages we will be using in the practical

In [ ]:
library(ggplot2)
library(plyr)
library(rms)
library(reshape2)
library(prodlim)
library(muhaz)
library(survival)
library(Hmisc)
library(MASS)

## Part 1
Load the data (smart) as we did in practical 1. The "Data/SMARTs_P2-3.rds" file is the same data as before but albumin is already a categorical variable and the SBP variable has already been generated

Begin by loading the data and conducting a brief data check

In [ ]:
smart <- readRDS("SMARTs_P2-3.rds")
attach(smart)
class(smart)
sapply(smart,class)
dim(smart)

In [ ]:
head(smart)

In [ ]:
sapply(smart,summary)

In [ ]:
summary(smart$TEVENT) # Summarises the time to event
table(smart$EVENT) # Tabulates number of events
table(smart$EVENT==1, round(smart$TEVENT/365,0)) # Tabulates number of events by year (TRUE=event, FALSE=censored)

## Part 2 
### Survival objects and Kaplan-Meier plots

We will now use R to illustrate 1) how to handle survival data, 2) how to illustrate it using the Kaplan-Meier estimate, and 3) how to build a cox proportional hazard regression model.

The columns of interest (for now) in the data are ‘TEVENT’ and ‘EVENT’.

‘survival’ is the R package that contains most of the functions we need to conduct survival analysis.
First, we need to tell R that ‘TEVENT’ and ‘EVENT’ are a pair that define the outcome, using a special function ‘Surv’:

In [ ]:
TE<-Surv(TEVENT,EVENT)

In [ ]:
head(TE)

This ‘Surv object’ is then used as our outcome in subsequent code. For example, we can construct a Kaplan Meier estimate:

In [ ]:
KM<-survfit(TE~1, data = smart)

This calculates the Kaplan-Meier estimate at all possible values of _t_ (note that the estimate only changes when events are observed). Note that ‘survfit’ we have specified a formula, and the ‘1’ means there are no covariates. More on this later.

We can also plot the Kaplan-Meier estimate. This will show the proportion of individuals alive at a given time.

In [ ]:
plot(KM,xlab="Time (days)",ylab="Survival")

We can also plot this in months (using TEVENT/12) or years (using TEVENT/365.25)

In [ ]:
TEmth<-Surv(TEVENT/12,EVENT)
KMmth<-survfit(TEmth~1, data = smart)
plot(KMmth,xlab="Time (months)",ylab="Survival", ylim = c(0.5,1))

Interpretation of the plot: we see a steady decline in the survival function over time. By the end of follow-up we estimate that approximatly 25% of the patients have had the event.

### Stratified Kaplan-Meier Estimates
We can also consider plotting survival estimates across different **categorical** variables. Producing Kaplan Meier estimates separately in the groups, and plotting these on the same graph. This is a good way to get a descriptive idea of the effect of the variable on time to event. We will start by looking at the association between gender:

In [ ]:
KM <-survfit(TE~SEX, data = smart)

plot(KM,lty=1:2,xlab="Time",ylab="Survival", ylim = c(0.5,1))

Here, we can see there there appears to be some difference in the survival curves, particularly at later follow-up. The log-rank test is a means to formally test the hypothesis that survival differs between the two groups. We can perform this test in R by:

In [ ]:
survdiff(TE~SEX, data = smart)

From the R output we see that P<0.001 and we therefore reject the null hypothesis that the survival curves are the same.

**By modifying the above code, explore the effect of other variables on survival. Remember, that you might need to categorise continuous variables**

In [ ]:
#Enter R code here

## Part 3
Now lets begin by fitting univariable models with categorical variables

In [ ]:
cox_sex <- coxph(Surv(TEVENT/12,EVENT)~SEX)
summary(cox_sex)

cox_card <- coxph(Surv(TEVENT/12,EVENT)~CARDIAC)
summary(cox_card)

The hazard ratio for the model including sex is 1.482 (95% confidence interval: 1.172-1.874). Hence, men have a 1.482 fold increase in their risk of experiencing the event compared to women. Pr(>|z|) (p-value) represents the probability that the model coefficient is zero.  
Note: this is not causal and is subject to confounding. Adjusting for additional predictors will likely change this estimate (see results at the end of the practical)

## Part 4
Modelling using splines  
Similar to logistic regression models we can investigate if a continuous variables should be modelled as a linear or non-linear association with with outcome of interest.

First create spline variables, where 3 means 3 knots (i.e. 2 age terms) etc.

In [ ]:
age3_spline <- rcs(AGE,3)
age4_spline <- rcs(AGE,4)
age5_spline <- rcs(AGE,5)


Fit a cox regression model using each set of spline variables & predict the linear predictor (LP). 

In [ ]:

# Fit a Cox model for each spline function
cox_age3 <- coxph(Surv(TEVENT/12,EVENT)~age3_spline,data=smart,ties="breslow")
cox_age3
lp_age3 <- predict(cox_age3)

cox_age4 <- coxph(Surv(TEVENT/12,EVENT)~age4_spline,data=smart,ties="breslow")
cox_age4
lp_age4 <- predict(cox_age4)

cox_age5 <- coxph(Surv(TEVENT/12,EVENT)~age5_spline,data=smart,ties="breslow")
cox_age5
lp_age5 <- predict(cox_age5)

We also need the non-spline model

In [ ]:
cox_age <- coxph(Surv(TEVENT/12,EVENT)~AGE)
summary(cox_age)
lp_age1 <- predict(cox_age)

Plot all LPs together for visual inspection

In [ ]:
data_part6 <- data.frame(AGE,lp_age1,lp_age3,lp_age4,lp_age5)
data_part6_m <- melt(data_part6,id.vars='AGE')
plot_part6 <- ggplot(data_part6_m,aes(AGE,value,colour=variable))+geom_line()+scale_colour_manual(labels=c("linear","3 knots","4 knots","5 knots"),values=c("gray","green","red","blue"))+theme_bw()
plot_part6 + labs(x="Age (years)",y="Linear Predictor (log odds)",color="") + theme(legend.position=c(0.2,0.8))

In [ ]:
age_spline_check <- matrix(c(AIC(cox_age),
         BIC(cox_age),
         AIC(cox_age3),
         BIC(cox_age3),
         AIC(cox_age4),
         BIC(cox_age4),
         AIC(cox_age5),
         BIC(cox_age5)), ncol=2, byrow=TRUE)

colnames(age_spline_check) <- c("AIC", "BIC")
rownames(age_spline_check) <- c("age_mod","age3_mod", "age4_mod", "age5_mod")
age_spline_check

**Using the plot of the spline functions and the table above, which age term would you use?**

**Now try the same but with BMI**

## Part 5 Building a multivariable model

Similar to practical 1, we will now select the same predictor variables of interest and build a multivariable cox model using stepwise selection.

In [ ]:
detach(smart)
smart <- subset(smart, select = c(outcome, TEVENT, EVENT, SEX, AGE, SBP, alcohol, CHOLO, BMIO, DIABETES, CARDIAC, SMOKING, AAA))
smart <- na.omit(smart)
age3_spline <- rcs(smart$AGE,3)
attach(smart)

In [ ]:

k10 <- qchisq(0.10,1,lower.tail=FALSE) # this gives the change in AIC we consider to be significant in our stepwise selection

# Forward selection (by AIC)
empty_mod_2 <- coxph(Surv(TEVENT/12,EVENT)~1)
forward_mod_2 <- stepAIC(empty_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~1),direction="forward",trace=TRUE)


In [ ]:
# Backward selection (by AIC)
full_mod_2 <- coxph(Surv(TEVENT/12,EVENT)~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA)
backward_mod_2 <- stepAIC(full_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~1),direction="backward",trace=TRUE)


In [ ]:
summary(forward_mod_2)
summary(backward_mod_2)

Remember the logistic model in practical 1 choose age, AAA and diabetes as significant predictors and we forced sex to be in the final model. Here, sex was chosen as a significant predictor in both the forward and backward selection approachs.  
Backward selection also included alcohol and smoking as predictors in the final model. As disscussed in practical 1, forward and backward selection do not always chose the same final model and we need to be careful when building our model.

For the stepwise selection we have been using a p-value of 0.1 (equivalent to an AIC of 2.706) so not to be too stringent. However, if we change this to 0.05 we can end with a different model. Lets try this in backward selection....

In [ ]:
k10 <- qchisq(0.05,1,lower.tail=FALSE)

full_mod_2 <- coxph(Surv(TEVENT/12,EVENT)~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA)
backward_mod_2a <- stepAIC(full_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~1),direction="backward",trace=TRUE)


Smoking and alcohol have not been included in this model. From the previous model we can see they are boarderline significant and so choosing a stringent p-value to build a model is often not the most appropriate as you can miss potentilly important predictors.  

In this practical we have built a model solely based on statistical results. However, in practice it is advised that both statistical and clinical relevance should be used to build a model. It is advised to build a model using clinical input and include variables that are clinically important even if the predictor does not appear statistically significant in the model

In [ ]:
full_mod_2 <- coxph(Surv(TEVENT/12,EVENT)~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA)
backward_mod_2a <- stepAIC(full_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~1),direction="backward",trace=TRUE)
